# Week 2 Assignment: CIFAR-10 Autoencoder

For this week, you will create a convolutional autoencoder for the [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset. You are free to choose the architecture of your autoencoder provided that the output image has the same dimensions as the input image.

After training, your model should meet loss and accuracy requirements when evaluated with the test dataset. You will then download the model and upload it in the classroom for grading. 

Let's begin!

***Important:*** *This colab notebook has read-only access so you won't be able to save your changes. If you want to save your work periodically, please click `File -> Save a Copy in Drive` to create a copy in your account, then work from there.*  

## Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10',split='train',as_supervised=True )

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10',split='test',as_supervised=True)

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

### END CODE HERE ###

## Build the Model

Create the autoencoder model. As shown in the lectures, you will want to downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Your input images will have the shape `(32, 32, 3)`. If you deviate from this, your model may not be recognized by the grader and may fail. 

We included a few hints to use the Sequential API below but feel free to remove it and use the Functional API just like in the ungraded labs if you're more comfortable with it. Another reason to use the latter is if you want to visualize the encoder output. As shown in the ungraded labs, it will be easier to indicate multiple outputs with the Functional API. That is not required for this assignment though so you can just stack layers sequentially if you want a simpler solution.

In [ ]:
from keras.layers import Conv2D, UpSampling2D,MaxPool2D


In [ ]:
class Autoencoder_Cifar(tf.keras.Model):
  def __init__(self):

    super(Autoencoder_Cifar,self).__init__()
    #encoder
    self.conv2d_1=Conv2D(64,3,activation='relu',padding='same')
    self.max_pooling_1=MaxPool2D((2,2))
    self.conv2d_2=Conv2D(128,3,activation='relu',padding='same')
    self.conv2d_3=Conv2D(256,3,activation='relu',padding='same')
    self.max_pooling_2=MaxPool2D((2,2))

    #decoder
    self.conv2d_1_dec=Conv2D(64,3,activation='relu',padding='same')
    self.conv2d_2_dec=Conv2D(128,3,activation='relu',padding='same')

    self.conv_out=Conv2D(3,3,activation='relu',padding='same')

    self.upsample_1=UpSampling2D((2,2))
    self.upsample_2=UpSampling2D((2,2))

  

  def encoder(self,inputs):
    x=self.conv2d_1(inputs)
    x=self.max_pooling_1(x)
    x=self.conv2d_2(x)
    x=self.max_pooling_2(x)
    out=self.conv2d_3(x)
    return out
  
  def decoder(self,inputs):
    x=self.conv2d_2_dec(inputs)
    x=self.upsample_1(x)
    x=self.conv2d_1_dec(x)
    x=self.upsample_2(x)
    out=self.conv_out(x)
    return out

  def call(self,inputs):


    encoder_out=self.encoder(inputs)
    decoder_out=self.decoder(encoder_out)

    return decoder_out


    


In [ ]:
def auto_encoder_model(input_shape):
  inputs=tf.keras.layers.Input(shape=input_shape)

  x=Conv2D(64,3,activation='relu',padding='same')(inputs)
  x=MaxPool2D((2,2))(x)
  x=Conv2D(128,3,activation='relu',padding='same')(x)
  x=MaxPool2D((2,2))(x)
  enc_out=Conv2D(256,3,activation='relu',padding='same')(x)

  x=Conv2D(128,3,activation='relu',padding='same')(enc_out)
  x=UpSampling2D((2,2))(x)
  x=Conv2D(64,3,activation='relu',padding='same')(x)
  x=UpSampling2D((2,2))(x)
  dec_out=Conv2D(3,3,activation='relu',padding='same')(x)

  return tf.keras.Model(inputs,enc_out),tf.keras.Model(inputs,dec_out)


In [ ]:
# suggested layers to use. feel free to add or remove as you see fit.

# use the Sequential API (you can remove if you want to use the Functional API)
enc_model,model = auto_encoder_model((32,32,3))
### START CODE HERE ###
# use `model.add()` to add layers (if using the Sequential API)







### END CODE HERE ###
model.summary()

In [ ]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')


In [ ]:
tf.keras.utils.plot_model(model)

## Configure training parameters

We have already provided the optimizer, metrics, and loss in the code below.

In [ ]:
# Please do not change the model.compile() parameters


## Training

You can now use [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method) to train your model. You will pass in the `train_dataset` and you are free to configure the other parameters. As with any training, you should see the loss generally going down and the accuracy going up with each epoch. If not, please revisit the previous sections to find possible bugs.

In [ ]:
len((train_dataset))

In [ ]:
# parameters
train_steps = len(train_dataset) 
val_steps = len(test_dataset)

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2,monitor='val_loss')
### START CODE HERE ###
H=model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=40)
### END CODE HERE ###

## Model evaluation

You can use this code to test your model locally before uploading to the grader. To pass, your model needs to satisfy these two requirements:

* loss must be less than 0.01 
* accuracy must be greater than 0.6

In [ ]:
result = model.evaluate(test_dataset, steps=10)

In [ ]:
enc_model.summary()

In [ ]:
samples = test_dataset.take(1)

# take the input images and put them in a list
output_samples = []
for input_image, image in tfds.as_numpy(test_dataset):
      output_samples = input_image[1:10]
      break


In [ ]:
len(output_samples)

In [ ]:
def display_one_row(disp_images, offset, shape=(28, 28)):
  '''Display sample outputs in one row.'''
  for idx, test_image in enumerate(disp_images):
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])
    test_image = np.reshape(test_image, shape)
    plt.imshow(test_image, cmap='gray')


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape=(8,4)):
  '''Displays the input, encoded, and decoded output values.'''
  plt.figure(figsize=(15, 5))
  display_one_row(disp_input_images, 0, shape=(32,32,3))
  display_one_row(disp_predicted, 20, shape=(32,32,3))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
display_results(output_samples,enc_model(output_samples),model(output_samples))

If you did some visualization like in the ungraded labs, then you might see something like the gallery below. This part is not required.

<img src="https://drive.google.com/uc?export=view&id=12Fy-guiP-3tTPfc9IV2nHhqLvs7LwRo6" width="75%" height="75%"/>

## Save your model

Once you are satisfied with the results, you can now save your model. Please download it from the Files window on the left and go back to the Submission portal in Coursera for grading.

In [ ]:
model.save('mymodel.h5')

**Congratulations on completing this week's assignment!**